# init
used to query DB for spectra and labels

In [8]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv

from pptoolbox.connectors import BaseEFSConnector, PFDBConnector

from datetime import datetime

In [9]:
load_dotenv(find_dotenv())

PF_SQL_PASSWORD = os.environ.get("PLATFORM_SQL_PASSWORD", None)  # PP_SQL_PASSWORD
PF_KEY_PATH = os.environ.get("DS_SERVER_KEYPATH", None)  # PP_SERVER_KEYPATH
PF_EFS_URL = os.environ.get("PF_EFS_URL", None)  # PP_EFS_URL

print(PF_SQL_PASSWORD,PF_KEY_PATH,PF_EFS_URL)

F398FYMLKN7XVVXJ C:/Users/RyanSoh/.ssh/datascience-research-instance-2.pem http://54.254.245.207:5014


In [10]:
raw_data_path = Path ("../data/raw")
output_folder = raw_data_path / "v1"
output_folder.mkdir(parents=True, exist_ok=True)

# by product_type

In [11]:
company_id = 1104
product_type = 6160
today_date = datetime.now().strftime("%y%m%d")

print (today_date)

260204


## label

In [12]:
BASE_QUERY = f"""
WITH combined_values AS (
    SELECT lnpv.lot_id, np.property_id, lnpv.value
    FROM lot_numerical_property_value lnpv
    JOIN numerical_property np ON lnpv.numerical_property_id = np.id
    
    UNION ALL
    
    SELECT lopv.lot_id, op.property_id, lopv.value
    FROM lot_option_property_value lopv
    JOIN option_property op ON lopv.option_property_id = op.id
)

SELECT 
    cv.lot_id, 
    l.name AS lot_name, 
    p.name AS property_name,
    cv.value AS property_value,
    l.company_id,
    l.product_type_id,
    pt.name AS product_name
FROM combined_values cv
INNER JOIN lot l ON l.id = cv.lot_id
INNER JOIN property p ON p.id = cv.property_id
INNER JOIN product_type pt ON pt.id = l.product_type_id
WHERE l.company_id = {company_id}
  AND l.product_type_id = {product_type}
ORDER BY cv.lot_id
"""

In [13]:
# BASE_QUERY = f"""
# SELECT lnpv.lot_id, lot.name as lot_name, 
# 	   property.name as property_name,
#        lnpv.value as property_value,
#        lot.company_id,
#        lot.product_type_id,
#        product_type.name as product_name
# FROM lot_numerical_property_value lnpv
# INNER JOIN lot on lot.id = lnpv.lot_id
# INNER JOIN numerical_property on numerical_property.id = lnpv.numerical_property_id
# INNER JOIN property on numerical_property.property_id = property.id
# INNER JOIN product_type on product_type.id = lot.product_type_id
# WHERE lot.company_id = {company_id}
# AND lot.product_type_id = {product_type}

# UNION ALL

# SELECT lopv.lot_id, lot.name as lot_name, 
# 	   property.name as property_name,
#        lopv.value as property_value,
#        lot.company_id,
#        lot.product_type_id,
#        product_type.name as product_name
# FROM lot_option_property_value lopv
# INNER JOIN lot on lot.id = lopv.lot_id
# INNER JOIN option_property on option_property.id = lopv.option_property_id
# INNER JOIN property on option_property.property_id = property.id
# INNER JOIN product_type on product_type.id = lot.product_type_id
# WHERE lot.company_id = {company_id}
# AND lot.product_type_id = {product_type}

# ORDER BY lot_id
# """

In [14]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY).set_index("lot_id")
print("successful query")

label_filename = output_folder / f"label_{today_date}.csv"
info_df.to_csv(label_filename)

successful query


## spectra

In [17]:
BASE_QUERY = f"""
SELECT
	sp.id AS specimen_id,
	l.id AS lot_id,
	l.name AS lot_name,
	sp.date_scanned,
	sp.analyzer_id AS analyzer_id,
    l.company_id as company_id,
	p.id AS product_id,
    p.name AS product_name,
	sp.frame_data AS has_nvd
FROM
	specimen sp
	INNER JOIN lot l ON l.id = sp.lot_id
	INNER JOIN product_type p on l.product_type_id = p.id
WHERE l.company_id = {company_id}
AND sp.active = 1
AND p.id = {product_type}
AND sp.date_scanned >= 1746748800
ORDER BY l.id;
"""

In [ ]:
SPECTRA_COLS = [
        "raw_data",
        "dark_ref_data",
        "white_ref_data",
        "dark_ref_scan_time",
        "white_ref_scan_time",
    ]
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY)
print("successful query")
efs_conn = BaseEFSConnector(url=PF_EFS_URL)
print(len(info_df))

nvd_df = efs_conn.get_nvd_specimens(info_df.specimen_id.values)

nvd_filename = output_folder / f"nvd_{today_date}.csv"
nvd_df.to_csv(nvd_filename)

successful query
4014


Fetching from EFS:  53%|█████▎    | 43/81 [06:57<05:59,  9.45s/it]